In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_top_n_names(frame, top_n):
    rank_se = frame.groupby("Name").max()["Elo"].rank(ascending=False, method='min')
    return pd.Series(rank_se[rank_se <= top_n].index)

In [ ]:
df = pd.read_csv('Master.csv', sep=';', encoding='utf-8', na_values='None')
df = df[df['Date'].notnull()]
df["Decade"] = df["Date"].str[:4].astype(int) // 10
print(df.shape)
# df.head()

In [ ]:
player_elo_df = pd.DataFrame(np.concatenate([df.loc[:, ["WhiteName", "WhiteElo", "Decade"]], df.loc[:, ["BlackName", "BlackElo", "Decade"]]]))
player_elo_df.columns = ['Name', 'Elo', 'Decade']
player_elo_df.dropna(inplace=True)
player_elo_df['Elo'] = player_elo_df['Elo'].astype(int)

In [ ]:
# Group by decade
g = player_elo_df.groupby("Decade")

In [ ]:
# For each decade group by name and get the max, then get the the names of the n highest ranked players
top_n = 100

apply_lambda = lambda frame: get_top_n_names(frame, top_n)
names_se = g.apply(apply_lambda)
unique_names_se = names_se.unique()

In [ ]:
# Get DataFrame with only top players and then write dataframe to csv
white_name_mask = df['WhiteName'].isin(unique_names_se)
black_name_mask = df['BlackName'].isin(unique_names_se)
full_mask = white_name_mask | black_name_mask

best_players_path = 'best_players.csv'
df[full_mask].to_csv(best_players_path, sep=';', index=False, encoding='utf-8', chunksize=1000)
print(df[full_mask].shape)
print(round(df[full_mask].shape[0] / df.shape[0], 5))